# Creating a Sampled Dataset

**Learning Objectives**

1. Setup up the environment
1. Sample the natality dataset to create train, eval, test sets
1. Preprocess the data in Pandas dataframe


## Introduction 
In this notebook, we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe for a small, repeatable sample.

We will set up the environment, sample the natality dataset to create train, eval, test splits, and preprocess the data in a Pandas dataframe.

## Set up environment variables and load necessary libraries

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
!pip install --user google-cloud-bigquery==1.25.0

     |████████████████████████████████| 169 kB 5.3 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tfx 0.23.0 requires attrs<20,>=19.3.0, but you'll have attrs 20.3.0 which is incompatible.
tfx 0.23.0 requires google-resumable-media<0.7.0,>=0.6.0, but you'll have google-resumable-media 0.5.1 which is incompatible.
tfx 0.23.0 requires kubernetes<12,>=10.0.1, but you'll have kubernetes 12.0.0 which is incompatible.
tfx 0.23.0 requires pyarrow<0.18,>=0.17, but you'll have pyarrow 2.0.0 which is incompatible.
google-cloud-storage 1.30.0 requires google-resumable-media<2.0dev,>=0.6.0, but you'll have google-resumable-media 0.5.1 which is incompatible.


**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

Import necessary libraries.

In [3]:
from google.cloud import bigquery
import pandas as pd

Set up environment variables so that we can use them throughout the notebook

In [11]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-02-47a2dd914f0d


In [17]:
PROJECT = "qwiklabs-gcp-02-47a2dd914f0d"  # Replace with your PROJECT

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets. Let's create a BigQuery client that we'll use throughout the demo.

In [18]:
bq = bigquery.Client(project = PROJECT)

We need to figure out the right way to divide our hash values to get our desired splits. To do that we need to define some values to hash within the module.

In [19]:
modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

We can make a series of queries to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly. Therefore, to make our code more compact and reusable, let's define a function to return the head of a dataframe produced from our queries up to a certain number of rows.

In [20]:
def display_dataframe_head_from_query(query, count=10):
    """Displays count rows from dataframe head from query.
    
    Args:
        query: str, query to be run on BigQuery, results stored in dataframe.
        count: int, number of results from head of dataframe to display.
    Returns:
        Dataframe head with count number of results.
    """
    df = bq.query(
        query + " LIMIT {limit}".format(
            limit=count)).to_dataframe()

    return df.head(count)

For our first query, we're going to use the original query above to get our label, features, and columns to combine into our hash which we will use to perform our repeatable splitting. There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are. We will need to include all of these extra columns to hash on to get a fairly uniform spread of the data. 

In [21]:
# Get label, features, and columns to hash and split into buckets
hash_cols_fixed_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
"""

display_dataframe_head_from_query(hash_cols_fixed_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,year,month,date,state,mother_birth_state
0,7.568469,True,22,1,46,2001,7,5,CA,CA
1,8.807467,True,39,1,42,2001,8,3,CA,Foreign
2,8.313632,True,23,1,35,2001,10,7,IL,IL
3,8.000575,False,27,1,40,2001,6,7,IL,IL
4,6.563162,False,29,1,39,2001,11,7,KY,IN
5,7.125340,False,34,1,40,2001,12,7,MD,MD
6,7.438397,False,31,1,38,2001,4,3,MA,Foreign
7,7.352416,True,30,1,37,2001,5,7,MI,MI
8,8.062305,True,16,1,40,2001,10,5,MN,MN
9,7.251004,True,17,1,39,2001,2,5,MS,MS


Using `COALESCE` would provide the same result as the nested `CASE WHEN`. This is preferable when all we want is the first non-null instance. To be precise the `CASE WHEN` would become `COALESCE(wday, day, 0) AS date`. You can read more about it [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/conditional_expressions).

Next query will combine our hash columns and will leave us just with our label, features, and our hash values.

In [22]:
data_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    ({CTE_hash_cols_fixed})
""".format(CTE_hash_cols_fixed=hash_cols_fixed_query)

display_dataframe_head_from_query(data_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,8.818490,False,17,1,42,6174834893904193287
1,8.141671,False,29,1,38,-4679450203487894236
2,5.948072,True,38,1,38,5830021534043063509
3,8.838332,True,27,1,39,-6484105101089659230
4,9.259415,True,28,1,38,-7561476152214523685
5,7.125340,True,34,1,39,4100584960946904090
6,7.063611,False,18,1,40,870533685982839311
7,7.500126,True,31,1,44,7999589519823312102
8,6.250105,False,23,1,41,-4634431482337591685
9,3.999185,False,30,1,32,-6873880094893634931


The next query is going to find the counts of each of the unique 657484 `hash_values`. This will be our first step at making actual hash buckets for our split via the `GROUP BY`.

In [23]:
# Get the counts of each of the unique hash of our splitting column
first_bucketing_query = """
SELECT
    hash_values,
    COUNT(*) AS num_records
FROM
    ({CTE_data})
GROUP BY
    hash_values
""".format(CTE_data=data_query)

display_dataframe_head_from_query(first_bucketing_query)

,hash_values,num_records
0,2341060194216507348,696
1,-8842767231851202242,515
2,7957807816914159435,369
3,-746421027886559730,167
4,-7566476151165360246,87
5,315818780995586851,400
6,8711610669332498583,7
7,-4170581330234584329,30
8,-4741381250325891292,1357
9,19168170974013054,503


The query below performs a second layer of bucketing where now for each of these bucket indices we count the number of records.

In [24]:
# Get the number of records in each of the hash buckets
second_bucketing_query = """
SELECT
    ABS(MOD(hash_values, {modulo_divisor})) AS bucket_index,
    SUM(num_records) AS num_records
FROM
    ({CTE_first_bucketing})
GROUP BY
    ABS(MOD(hash_values, {modulo_divisor}))
""".format(
    CTE_first_bucketing=first_bucketing_query, modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(second_bucketing_query)

,bucket_index,num_records
0,1,163893
1,2,492473
2,3,196889
3,93,215710
4,37,295153
5,30,333513
6,49,468121
7,90,286465
8,80,312489
9,91,333267


The number of records is hard for us to easily understand the split, so we will normalize the count into percentage of the data in each of the hash buckets in the next query.

In [25]:
# Calculate the overall percentages
percentages_query = """
SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
        SUM(num_records)
    FROM
        ({CTE_second_bucketing})) AS percent_records
FROM
    ({CTE_second_bucketing})
""".format(CTE_second_bucketing=second_bucketing_query)

display_dataframe_head_from_query(percentages_query)

,bucket_index,num_records,percent_records
0,36,246041,0.007453
1,85,368045,0.011149
2,98,374697,0.011351
3,97,480790,0.014564
4,3,196889,0.005964
5,92,336735,0.010201
6,67,372457,0.011283
7,5,449280,0.013610
8,35,250505,0.007588
9,96,529357,0.016036


We'll now select the range of buckets to be used in training.

In [26]:
# Choose hash buckets for training and pull in their statistics
train_query = """
SELECT
    *,
    "train" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= 0
    AND bucket_index < {train_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets)

display_dataframe_head_from_query(train_query)

,bucket_index,num_records,percent_records,dataset_name
0,58,209618,0.006350,train
1,72,229541,0.006953,train
2,65,289303,0.008764,train
3,48,370308,0.011218,train
4,13,274104,0.008303,train
5,50,184434,0.005587,train
6,60,355099,0.010757,train
7,62,426834,0.012930,train
8,79,403701,0.012229,train
9,12,412875,0.012507,train


We'll do the same by selecting the range of buckets to be used evaluation.

In [27]:
# Choose hash buckets for validation and pull in their statistics
eval_query = """
SELECT
    *,
    "eval" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {train_buckets}
    AND bucket_index < {cum_eval_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets,
    cum_eval_buckets=train_buckets + eval_buckets)

display_dataframe_head_from_query(eval_query)

,bucket_index,num_records,percent_records,dataset_name
0,85,368045,0.011149,eval
1,87,523881,0.015870,eval
2,89,256482,0.007770,eval
3,82,468179,0.014182,eval
4,83,411258,0.012458,eval
5,81,233538,0.007074,eval
6,80,312489,0.009466,eval
7,88,423809,0.012838,eval
8,86,274489,0.008315,eval
9,84,341155,0.010334,eval


Lastly, we'll select the hash buckets to be used for the test split.

In [28]:
# Choose hash buckets for testing and pull in their statistics
test_query = """
SELECT
    *,
    "test" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {cum_eval_buckets}
    AND bucket_index < {modulo_divisor}
""".format(
    CTE_percentages=percentages_query,
    cum_eval_buckets=train_buckets + eval_buckets,
    modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(test_query)

,bucket_index,num_records,percent_records,dataset_name
0,90,286465,0.008678,test
1,97,480790,0.014564,test
2,94,431001,0.013056,test
3,91,333267,0.010096,test
4,99,223334,0.006765,test
5,96,529357,0.016036,test
6,93,215710,0.006534,test
7,95,313544,0.009498,test
8,92,336735,0.010201,test
9,98,374697,0.011351,test


In the below query, we'll `UNION ALL` all of the datasets together so that all three sets of hash buckets will be within one table. We added `dataset_id` so that we can sort on it in the query after.

In [29]:
# Union the training, validation, and testing dataset statistics
union_query = """
SELECT
    0 AS dataset_id,
    *
FROM
    ({CTE_train})
UNION ALL
SELECT
    1 AS dataset_id,
    *
FROM
    ({CTE_eval})
UNION ALL
SELECT
    2 AS dataset_id,
    *
FROM
    ({CTE_test})
""".format(CTE_train=train_query, CTE_eval=eval_query, CTE_test=test_query)

display_dataframe_head_from_query(union_query)

,dataset_id,bucket_index,num_records,percent_records,dataset_name
0,0,60,355099,0.010757,train
1,0,46,281627,0.008531,train
2,0,75,367455,0.011131,train
3,0,38,338150,0.010243,train
4,0,15,263367,0.007978,train
5,0,20,432535,0.013103,train
6,0,6,548778,0.016624,train
7,0,59,361078,0.010938,train
8,0,21,247072,0.007484,train
9,0,62,426834,0.012930,train


Lastly, we'll show the final split between train, eval, and test sets. We can see both the number of records and percent of the total data. It is really close to that we were hoping to get.

In [30]:
# Show final splitting and associated statistics
split_query = """
SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
FROM
    ({CTE_union})
GROUP BY
    dataset_id,
    dataset_name
ORDER BY
    dataset_id
""".format(CTE_union=union_query)

display_dataframe_head_from_query(split_query)

,dataset_id,dataset_name,num_records,percent_records
0,0,train,25873134,0.783765
1,1,eval,3613325,0.109457
2,2,test,3524900,0.106778


Now that we know that our splitting values produce a good global splitting on our data, here's a way to get a well-distributed portion of the data in such a way that the train, eval, test sets do not overlap and takes a subsample of our global splits.

Sample the natality dataset

In [31]:
# every_n allows us to subsample from each of the hash values
# This helps us get approximately the record counts we want
every_n = 1000

splitting_string = "ABS(MOD(hash_values, {0} * {1}))".format(every_n, modulo_divisor)

def create_data_split_sample_df(query_string, splitting_string, lo, up):
    """Creates a dataframe with a sample of a data split.

    Args:
        query_string: str, query to run to generate splits.
        splitting_string: str, modulo string to split by.
        lo: float, lower bound for bucket filtering for split.
        up: float, upper bound for bucket filtering for split.
    Returns:
        Dataframe containing data split sample.
    """
    query = "SELECT * FROM ({0}) WHERE {1} >= {2} and {1} < {3}".format(
        query_string, splitting_string, int(lo), int(up))

    df = bq.query(query).to_dataframe()

    return df

train_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=0, up=train_percent)

eval_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent, up=train_percent + eval_percent)

test_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent + eval_percent, up=modulo_divisor)

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 7733 examples in the train dataset.
There are 1037 examples in the validation dataset.
There are 561 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, we'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [32]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,6.977631,False,38,1,38,-6784884401981100070
1,7.561856,False,34,1,40,4896699230184800022
2,6.161920,True,33,1,36,-7200461189606400005
3,6.406633,True,33,1,41,-6784884401981100070
4,7.874912,True,29,1,39,4896699230184800022


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [33]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hash_values
count,7733.000000,7733.000000,7733.000000,7733.000000,7.733000e+03
mean,7.264415,28.213371,1.035691,38.691064,-2.984870e+17
std,1.303220,6.134232,0.201568,2.531921,5.590715e+18
min,0.562179,13.000000,1.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,1.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,1.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,1.000000,40.000000,4.896699e+18
max,11.563246,48.000000,4.000000,47.000000,9.203641e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

Preprocess the data in Pandas dataframe

In [34]:
  def preprocess(df):
    """ Preprocess pandas dataframe for augmented babyweight data.
    
    Args:
        df: Dataframe containing raw babyweight data.
    Returns:
        Pandas dataframe containing preprocessed raw babyweight data as well
            as simulated no ultrasound data masking some of the original data.
    """
    # Clean up raw data
    # Filter out what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)",
                    "Twins(2)",
                    "Triplets(3)",
                    "Quadruplets(4)",
                    "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace=True)

    # Clone data and mask certain columns to simulate lack of ultrasound
    no_ultrasound = df.copy(deep=True)

    # Modify is_male
    no_ultrasound["is_male"] = "Unknown"
    
    # Modify plurality
    condition = no_ultrasound["plurality"] != "Single(1)"
    no_ultrasound.loc[condition, "plurality"] = "Multiple(2+)"

    # Concatenate both datasets together and shuffle
    return pd.concat(
        [df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train, eval, test set and see a small sample of the training data after our preprocessing:

In [35]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [36]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,5.996574,True,35,Single(1),37,-4761448277899300037
1,7.683110,False,40,Single(1),40,-8492389082673800074
2,7.861684,True,31,Single(1),39,-6784884401981100070
3,6.937947,True,30,Single(1),39,6820283424305200067
4,3.126155,True,36,Single(1),40,-7821205650511500030


In [37]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
15461,8.247493,False,28,Single(1),40,-6784884401981100070
15462,6.874013,Unknown,22,Single(1),45,8655151740159000017
15463,8.289381,Unknown,19,Single(1),45,505732274561700014
15464,7.687519,Unknown,30,Single(1),39,-7821205650511500030
15465,6.437498,True,36,Single(1),38,766709067980000060


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [38]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hash_values
count,15466.000000,15466.000000,15466.000000,1.546600e+04
mean,7.264415,28.213371,38.691064,-2.984870e+17
std,1.303178,6.134034,2.531839,5.590534e+18
min,0.562179,13.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,40.000000,4.896699e+18
max,11.563246,48.000000,47.000000,9.203641e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [39]:
# Define columns
columns = ["weight_pounds",
           "is_male",
           "mother_age",
           "plurality",
           "gestation_weeks"]

# Write out CSV files
train_df.to_csv(
    path_or_buf="train.csv", columns=columns, header=False, index=False)
eval_df.to_csv(
    path_or_buf="eval.csv", columns=columns, header=False, index=False)
test_df.to_csv(
    path_or_buf="test.csv", columns=columns, header=False, index=False)

In [40]:
%%bash
wc -l *.csv

  2074 eval.csv
  1122 test.csv
 15466 train.csv
 18662 total


In [41]:
%%bash
head *.csv

==> eval.csv <==
6.75055446244,True,18,Single(1),36
8.811876612139999,Unknown,28,Single(1),39
7.16061426976,False,32,Single(1),38
7.5618555866,True,33,Single(1),39
7.55524171874,Unknown,25,Single(1),41
9.37626000286,False,35,Single(1),40
7.7272022831,Unknown,28,Single(1),41
5.5336027761999995,Unknown,33,Single(1),35
5.24920645822,Unknown,31,Single(1),39
8.1240343547,Unknown,27,Single(1),41

==> test.csv <==
9.12493302418,Unknown,23,Single(1),41
6.7020527647999995,False,35,Single(1),36
7.87491199864,Unknown,24,Single(1),40
7.50012615324,True,22,Single(1),38
8.41504454054,Unknown,32,Single(1),40
6.6248909731,False,21,Single(1),39
8.62448368944,False,18,Single(1),42
6.5807985207,Unknown,20,Single(1),40
8.313631900019999,Unknown,28,Single(1),40
8.8956522717,Unknown,19,Single(1),46

==> train.csv <==
5.9965735264,True,35,Single(1),37
7.683109830699999,False,40,Single(1),40
7.86168426292,True,31,Single(1),39
6.93794738514,True,30,Single(1),39
3.12615487516,True,36,Single(1),40
6.93794738514,

In [42]:
%%bash
tail *.csv

==> eval.csv <==
6.0009827716399995,True,22,Single(1),37
6.4992274837599995,False,40,Twins(2),37
9.43798943622,False,29,Single(1),42
8.375361333379999,False,23,Single(1),40
8.50102482272,False,34,Single(1),40
7.5618555866,Unknown,33,Single(1),40
7.31273323054,Unknown,32,Single(1),36
8.18796841068,Unknown,32,Single(1),39
7.62578964258,Unknown,27,Single(1),37
6.8122838958,Unknown,29,Single(1),37

==> test.csv <==
6.686620406459999,False,25,Single(1),40
9.3586230219,True,22,Single(1),40
6.2501051276999995,True,34,Single(1),40
7.5618555866,Unknown,16,Single(1),40
6.8122838958,Unknown,25,Single(1),37
7.50012615324,False,22,Single(1),39
6.4992274837599995,Unknown,28,Single(1),39
7.31273323054,False,25,Single(1),36
7.62578964258,False,23,Single(1),40
7.06361087448,Unknown,35,Single(1),37

==> train.csv <==
5.24920645822,Unknown,19,Single(1),38
7.04817851614,Unknown,40,Single(1),38
7.09227096854,True,30,Single(1),40
8.24969784404,False,25,Single(1),38
8.62448368944,False,42,Single(1),40
8.2474

## Lab Summary: 
In this lab, we set up the environment, sampled the natality dataset to create train, eval, test splits, and preprocessed the data in a Pandas dataframe.